In [ ]:
import torch
import os 
import random
import shutil
import xml.etree.ElementTree as ET


from sklearn.model_selection import train_test_split
from xml.dom import minidom
from tqdm import tqdm

In [2]:
def xml2dic(xml_file):
    data = ET.parse(xml_file).getroot()
    # print("data")

    data_dict = {}
    data_dict['bounding_box'] = []

    for elem in data:
        if elem.tag == "filename":
            data_dict['filename'] = elem.text
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            data_dict['image_size'] = tuple(image_size)
        elif elem.tag == "object":
            bb = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bb["class"] = subelem.text
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bb[subsubelem.tag] = int(subsubelem.text)            
            data_dict['bounding_box'].append(bb)
    
    return data_dict

In [ ]:
# import pandas
# pandas.DataFrame(xml2dic('Road_Sign_Dataset/annotations/road8.xml'))

In [4]:
import shutil

class_dict = {"crosswalk": 0}

def dict2yolo(data_dict):
    # print_buffer = []

    for b in data_dict["bounding_box"]:
        if b["class"] ==  'crosswalk':
            class_id = class_dict[b["class"]]

            center_x = (b["xmin"] + b["xmax"]) / 2 
            center_y = (b["ymin"] + b["ymax"]) / 2
            width    = (b["xmax"] - b["xmin"])
            height   = (b["ymax"] - b["ymin"])
            
            img_w, img_h, img_depth = data_dict["image_size"]  
            center_x = center_x / img_w 
            center_y = center_y / img_h 
            width    = width / img_w 
            height   = height / img_h 
            
            print_buffer = []
            print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, center_x, center_y, width, height))  
            save_file_name = os.path.join("Road_Sign_Dataset/labels", data_dict["filename"].replace("png", "txt"))
            print("\n".join(print_buffer), file= open(save_file_name, "w"))

            img_path = os.path.join("images", data_dict["filename"])
            src = os.path.abspath(img_path)

            img_path = os.path.join("Road_Sign_Dataset/images", data_dict["filename"])
            dst = os.path.abspath(img_path)

            shutil.copyfile(src, dst)
        # else:
        #     xml_name = data_dict["filename"][0:-3] + 'xml'

        #     ano_path = os.path.join("Road_Sign_Dataset/annotations", xml_name)
        #     img_path = os.path.join("Road_Sign_Dataset/images", data_dict["filename"])

        #     ano_abs_path = os.path.abspath(ano_path)
        #     img_abs_path = os.path.abspath(img_path)

        #     os.remove(ano_abs_path) 
        #     os.remove(img_abs_path) 

        
    # save_file_name = os.path.join("Road_Sign_Dataset/annotations", data_dict["filename"].replace("png", "txt"))
    # print("\n".join(print_buffer), file= open(save_file_name, "w"))


In [5]:

ano_path = os.path.join("annotations", 'road147.xml')
img_path = os.path.join("images", 'road147.png')
os.path.abspath(img_path)

'/home/grzegorz/Documents/Politechnika/Semestr_5/WDSI/projekt_V2/Road_Sign_Dataset/images/road147.png'

In [14]:
ann_list = []

for ann in os.listdir('annotations'):
    if ann[-3:] == "xml":
        ann_list.append(ann)


annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "xml"]
annotations.sort()

for ann in tqdm(annotations):
    info_dict = xml2dic(ann)
    dict2yolo(info_dict)

annotations = [os.path.join('Road_Sign_Dataset/labels', x) for x in os.listdir('Road_Sign_Dataset/labels') if x[-3:] == "txt"]

100%|██████████| 877/877 [00:00<00:00, 4881.34it/s]


In [18]:
images = [os.path.join('Road_Sign_Dataset/images', x) for x in os.listdir('Road_Sign_Dataset/images')]
labels = [os.path.join('Road_Sign_Dataset/labels', x) for x in os.listdir('Road_Sign_Dataset/labels') if x[-3:] == "txt"]

images.sort()
labels.sort()


train_images, val_images, train_annotations, val_annotations = train_test_split(images, labels, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
!mkdir images/train images/val images/test labels/train labels/val labels/test

In [13]:
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        shutil.move(f, destination_folder)

move_files_to_folder(train_images, 'Road_Sign_Dataset/images/train')
move_files_to_folder(val_images, 'Road_Sign_Dataset/images/val/')
move_files_to_folder(test_images, 'Road_Sign_Dataset/images/test/')
move_files_to_folder(train_annotations, 'Road_Sign_Dataset/labels/train/')
move_files_to_folder(val_annotations, 'Road_Sign_Dataset/labels/val/')
move_files_to_folder(test_annotations, 'Road_Sign_Dataset/labels/test/')

In [ ]:
# python3 train.py --img 120 --cfg yolov5s.yaml --hyp hyp.scratch.yaml --batch 12 --epochs 15 --data road_sign_data.yaml --weights yolov5s.pt --workers 24 --name yolo_road_det

In [ ]:
# python3 detect.py --source ../Road_Sign_Dataset/images/test/ --weights runs/train/yolo_road_det25/weights/best.pt --conf 0.25 --name yolo_road_det --nosave 
